In [17]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
#from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
#import xgboost as xgb

In [18]:
from imblearn.over_sampling import RandomOverSampler

In [16]:
pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\posidex\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
df=pd.read_excel(r"C:\Users\posidex\Desktop\Regression\Credit_card\notebooks\data\default of credit card clients.xls")
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
df.SEX = df.SEX.astype("category")
df.EDUCATION = df.EDUCATION.astype("category")
df.MARRIAGE = df.MARRIAGE.astype("category")

In [5]:
df.dtypes

LIMIT_BAL                        int64
SEX                           category
EDUCATION                     category
MARRIAGE                      category
AGE                              int64
PAY_0                            int64
PAY_2                            int64
PAY_3                            int64
PAY_4                            int64
PAY_5                            int64
PAY_6                            int64
BILL_AMT1                        int64
BILL_AMT2                        int64
BILL_AMT3                        int64
BILL_AMT4                        int64
BILL_AMT5                        int64
BILL_AMT6                        int64
PAY_AMT1                         int64
PAY_AMT2                         int64
PAY_AMT3                         int64
PAY_AMT4                         int64
PAY_AMT5                         int64
PAY_AMT6                         int64
default payment next month       int64
dtype: object

In [6]:
df = df.drop_duplicates()

In [8]:
X = df.drop(labels=['default payment next month'],axis=1)
Y = df[['default payment next month']]

In [11]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [12]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [49]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('scaler',StandardScaler()),

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
('ordinalencoder',OrdinalEncoder()),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])




In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=30)

# Apply the preprocessing to the features
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Apply oversampling to the target variable (y_train) to balance the classes
oversampler = RandomOverSampler(random_state=42)  # You can adjust the random_state as needed
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_processed, y_train)


array([[-0.82616892,  0.8131756 ,  1.46224782, ...,  0.98075688,
        -0.28535525, -0.2848968 ],
       [-0.98102989,  0.8131756 ,  0.18685494, ..., -0.20630871,
        -0.19464084, -0.18873761],
       [ 1.03216267,  0.8131756 ,  1.46224782, ..., -0.30514884,
        -0.31127365, -0.2848968 ],
       ...,
       [ 1.10959315,  0.8131756 ,  0.18685494, ...,  0.10079069,
         0.14229838,  0.12659423],
       [-0.98102989,  0.8131756 ,  0.18685494, ..., -0.28054644,
        -0.31127365, -0.16408   ],
       [-1.05846037,  0.8131756 ,  1.46224782, ..., -0.17506367,
        -0.31127365, -0.17506334]])

In [54]:
X_train=pd.DataFrame(X_train_resampled,columns=preprocessor.get_feature_names_out())

In [55]:
X_train

,num_pipeline__LIMIT_BAL,num_pipeline__SEX,num_pipeline__EDUCATION,num_pipeline__MARRIAGE,num_pipeline__AGE,num_pipeline__PAY_0,num_pipeline__PAY_2,num_pipeline__PAY_3,num_pipeline__PAY_4,num_pipeline__PAY_5,...,num_pipeline__BILL_AMT3,num_pipeline__BILL_AMT4,num_pipeline__BILL_AMT5,num_pipeline__BILL_AMT6,num_pipeline__PAY_AMT1,num_pipeline__PAY_AMT2,num_pipeline__PAY_AMT3,num_pipeline__PAY_AMT4,num_pipeline__PAY_AMT5,num_pipeline__PAY_AMT6
0,-0.826169,0.813176,1.462248,2.767900,1.361029,0.017490,0.108337,1.814999,1.908143,-0.652384,...,0.105852,-0.686444,-0.337166,-0.321895,-0.126278,-0.269306,-0.317957,0.980757,-0.285355,-0.284897
1,-0.981030,0.813176,0.186855,0.856065,-1.242796,-1.763329,-1.564982,-1.541887,-1.532206,-1.536633,...,-0.140304,-0.079512,-0.028644,-0.030333,-0.231669,-0.194945,-0.231679,-0.206309,-0.194641,-0.188738
2,1.032163,0.813176,1.462248,0.856065,-0.591840,-1.763329,-1.564982,-1.541887,-1.532206,-1.536633,...,-0.678583,-0.677656,-0.670656,-0.659840,-0.356465,-0.269306,-0.317957,-0.305149,-0.311274,-0.284897
3,0.335288,0.813176,-1.088538,-1.055771,0.384594,0.017490,0.108337,0.136556,0.187968,0.231865,...,-0.278707,-0.201165,-0.286351,0.503401,0.267519,-0.205542,-0.010043,-0.224945,3.576487,-0.120147
4,0.877302,0.813176,0.186855,-1.055771,0.818565,0.907899,-0.728323,-0.702666,-0.672119,-0.652384,...,-0.661350,-0.366319,-0.660725,-0.645478,-0.103689,-0.211444,0.906001,-0.268307,-0.256197,-0.274737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32801,-0.516447,0.813176,-1.088538,0.856065,-0.808826,1.798308,0.108337,0.136556,0.187968,0.231865,...,-0.534964,-0.368966,-0.332806,-0.287458,-0.169270,-0.129880,0.297871,-0.243643,-0.181682,-0.065230
32802,-0.284156,-1.229747,0.186855,-1.055771,-0.483347,1.798308,1.781656,0.136556,0.187968,0.231865,...,1.162194,0.893380,0.880358,0.954203,-0.356465,-0.032282,-0.065467,-0.096521,-0.083127,-0.080387
32803,1.109593,0.813176,0.186855,0.856065,-0.483347,0.017490,0.108337,0.136556,0.187968,0.231865,...,3.750462,4.013961,2.939672,3.018001,0.329917,0.311635,0.359454,0.100791,0.142298,0.126594
32804,-0.981030,0.813176,0.186855,0.856065,-1.025811,1.798308,1.781656,1.814999,1.908143,2.000363,...,-0.540368,-0.514225,-0.496226,-0.487437,-0.075673,-0.269306,-0.256374,-0.280546,-0.311274,-0.164080


In [56]:
logreg = LogisticRegression(random_state=2)
logreg.fit(X_train, y_train_resampled)

LogisticRegression(random_state=2)

In [59]:
X_test_processed 

array([[ 0.4127188 , -1.22974669,  0.18685494, ..., -0.18158331,
        -0.21608832, -0.17506334],
       [ 0.25785784, -1.22974669, -1.08853794, ..., -0.14080484,
        -0.31127365, -0.18264185],
       [ 1.80646749, -1.22974669, -1.08853794, ...,  0.74020696,
         0.01335433, -0.27962479],
       ...,
       [ 0.95473218,  0.8131756 ,  0.18685494, ...,  0.06425613,
         0.04516917,  0.54236877],
       [ 0.25785784, -1.22974669,  1.46224782, ..., -0.30514884,
        -0.18621736,  0.01439936],
       [-0.12929458,  0.8131756 ,  0.18685494, ..., -0.18213686,
        -0.18168164, -0.17506334]])

In [60]:
y_pred=logreg.predict(X_test_processed)

In [61]:
#### Model Evaluation using Confusion Matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix


array([[4894, 2038],
       [ 729, 1329]], dtype=int64)

In [63]:
cf=classification_report(y_test, y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.87      0.71      0.78      6932
           1       0.39      0.65      0.49      2058

    accuracy                           0.69      8990
   macro avg       0.63      0.68      0.63      8990
weighted avg       0.76      0.69      0.71      8990



In [67]:
dt = DecisionTreeClassifier(random_state=4)
dt.fit(X_train, y_train_resampled)
y_pred=dt.predict(X_test_processed)
cnf_matrix = confusion_matrix(y_test, y_pred)
cf=classification_report(y_test, y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      6932
           1       0.39      0.40      0.39      2058

    accuracy                           0.72      8990
   macro avg       0.60      0.61      0.60      8990
weighted avg       0.72      0.72      0.72      8990



In [ ]:
classifiers = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier())
]


In [ ]:
models = {
    "Logistic classification ": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

In [46]:
)

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. 'Pipeline(steps=[('scaler', StandardScaler()),
                ('oversampler', RandomOverSampler())])' (type <class 'sklearn.pipeline.Pipeline'>) doesn't.